In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import os
from pprint import pprint
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import psycopg2
from sqlalchemy import create_engine
import sys
import numpy as np

In [2]:
sys.path.insert(0, os.path.realpath('..\src'))
from database.database_config import DB_NAME, DB_USER, DB_PASSWORD, DB_HOST

In [3]:
db = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}")
conn = db.connect()

In [4]:
scores_overall_df = pd.read_sql("select * from laligadb.laliga.scores_overall order by _date_", con=conn)

In [5]:
scores_overall_df

,_week_,_day_,_date_,_time_,home,score,away,attendance,venue,referee,xg_home,xg_away
0,1,Sat,2014-08-23,19:00:00,Málaga,1–0,Athletic Club,24500.0,Estadio La Rosaleda,Antonio Matéu Lahoz,NaN,NaN
1,1,Sat,2014-08-23,23:00:00,Almería,1–1,Espanyol,12000.0,Estadio de los Juegos Mediterráneos,Alejandro Hernández,NaN,NaN
2,1,Sat,2014-08-23,21:00:00,Granada,2–1,La Coruña,14800.0,Estadio Nuevo Los Cármenes,Iñaki Bikandi,NaN,NaN
3,1,Sat,2014-08-23,21:00:00,Sevilla,1–1,Valencia,29352.0,Estadio Ramón Sánchez Pizjuán,Carlos del Cerro,NaN,NaN
4,1,Sun,2014-08-24,19:00:00,Eibar,1–0,Real Sociedad,5173.0,Estadio Municipal de Ipurúa,Alfonso Álvarez,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3145,11,Sun,2022-10-23,14:00:00,Espanyol,None,Elche,NaN,RCDE Stadium,None,NaN,NaN
3146,11,Sun,2022-10-23,16:15:00,Betis,None,Atlético Madrid,NaN,Estadio Benito Villamarín,None,NaN,NaN
3147,11,Sun,2022-10-23,18:30:00,Villarreal,None,Almería,NaN,Estadio Ciudad de Valencia,None,NaN,NaN
3148,11,Sun,2022-10-23,21:00:00,Barcelona,None,Athletic Club,NaN,Camp Nou,None,NaN,NaN


In [6]:
scores_overall_df.dtypes

_week_          int64
_day_          object
_date_         object
_time_         object
home           object
score          object
away           object
attendance    float64
venue          object
referee        object
xg_home       float64
xg_away       float64
dtype: object

In [7]:
scores_overall_df.dtypes

_week_          int64
_day_          object
_date_         object
_time_         object
home           object
score          object
away           object
attendance    float64
venue          object
referee        object
xg_home       float64
xg_away       float64
dtype: object

In [8]:
start_date = pd.to_datetime('2017-08-01').date()
end_date = pd.to_datetime('2022-05-30').date()
mask = (scores_overall_df['_date_'] > start_date) & (scores_overall_df['_date_'] <= end_date)
scores_overall_df = scores_overall_df.loc[mask]
scores_overall_df

,_week_,_day_,_date_,_time_,home,score,away,attendance,venue,referee,xg_home,xg_away
1140,1,Fri,2017-08-18,20:15:00,Leganés,1–0,Alavés,9231.0,Estadio Municipal de Butarque,José Munuera,1.4,1.2
1141,1,Fri,2017-08-18,22:15:00,Valencia,1–0,Las Palmas,35971.0,Estadio de Mestalla,Jesús Gil,1.9,0.3
1142,1,Sat,2017-08-19,18:15:00,Celta Vigo,2–3,Real Sociedad,16961.0,Estadio de Balaídos,Antonio Matéu Lahoz,1.6,2.4
1143,1,Sat,2017-08-19,20:15:00,Girona,2–2,Atlético Madrid,11511.0,Estadi Municipal de Montilivi,Juan Martínez,2.1,0.8
1144,1,Sat,2017-08-19,22:15:00,Sevilla,1–1,Espanyol,30487.0,Estadio Ramón Sánchez Pizjuán,Alejandro Hernández,2.1,1.1
...,...,...,...,...,...,...,...,...,...,...,...,...
3035,38,Sun,2022-05-22,22:00:00,Sevilla,1–0,Athletic Club,31305.0,Estadio Ramón Sánchez Pizjuán,Carlos del Cerro,1.4,0.7
3036,38,Sun,2022-05-22,22:00:00,Real Sociedad,1–2,Atlético Madrid,23586.0,Estadio Municipal de Anoeta,Jesús Gil,2.3,0.8
3037,38,Sun,2022-05-22,20:00:00,Granada,0–0,Espanyol,17951.0,Estadio Nuevo Los Cármenes,Alejandro Hernández,2.0,0.4
3038,38,Sun,2022-05-22,17:30:00,Elche,3–1,Getafe,17336.0,Estadio Manuel Martínez Valero,Guillermo Cuadra,2.5,1.3


In [9]:
scores_overall_df['_date_'] = pd.to_datetime(scores_overall_df['_date_'], format = "%Y-%m-%d")
scores_overall_df['_time_'] = pd.to_datetime(scores_overall_df['_time_'],format = "%H:%M:%S").dt.time
scores_overall_df['attendance'] = pd.to_numeric(scores_overall_df['attendance'], errors = 'coerce', downcast= 'integer').astype('Int64')
scores_overall_df[['homegoal', 'awaygoal']] = scores_overall_df.score.str.split('–',expand = True)
scores_overall_df['homegoal'] = pd.to_numeric(scores_overall_df['homegoal'], errors = 'coerce', downcast= 'integer')
scores_overall_df['awaygoal'] = pd.to_numeric(scores_overall_df['awaygoal'], errors = 'coerce', downcast= 'integer')

C:\Users\ntlg4\AppData\Local\Temp\ipykernel_29012\1711038850.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scores_overall_df['_date_'] = pd.to_datetime(scores_overall_df['_date_'], format = "%Y-%m-%d")
C:\Users\ntlg4\AppData\Local\Temp\ipykernel_29012\1711038850.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scores_overall_df['_time_'] = pd.to_datetime(scores_overall_df['_time_'],format = "%H:%M:%S").dt.time
C:\Users\ntlg4\AppData\Local\Temp\ipykernel_29012\1711038850.py:3: SettingWithCopyWarning

In [10]:
scores_overall_df

,_week_,_day_,_date_,_time_,home,score,away,attendance,venue,referee,xg_home,xg_away,homegoal,awaygoal
1140,1,Fri,2017-08-18,20:15:00,Leganés,1–0,Alavés,9231,Estadio Municipal de Butarque,José Munuera,1.4,1.2,1,0
1141,1,Fri,2017-08-18,22:15:00,Valencia,1–0,Las Palmas,35971,Estadio de Mestalla,Jesús Gil,1.9,0.3,1,0
1142,1,Sat,2017-08-19,18:15:00,Celta Vigo,2–3,Real Sociedad,16961,Estadio de Balaídos,Antonio Matéu Lahoz,1.6,2.4,2,3
1143,1,Sat,2017-08-19,20:15:00,Girona,2–2,Atlético Madrid,11511,Estadi Municipal de Montilivi,Juan Martínez,2.1,0.8,2,2
1144,1,Sat,2017-08-19,22:15:00,Sevilla,1–1,Espanyol,30487,Estadio Ramón Sánchez Pizjuán,Alejandro Hernández,2.1,1.1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035,38,Sun,2022-05-22,22:00:00,Sevilla,1–0,Athletic Club,31305,Estadio Ramón Sánchez Pizjuán,Carlos del Cerro,1.4,0.7,1,0
3036,38,Sun,2022-05-22,22:00:00,Real Sociedad,1–2,Atlético Madrid,23586,Estadio Municipal de Anoeta,Jesús Gil,2.3,0.8,1,2
3037,38,Sun,2022-05-22,20:00:00,Granada,0–0,Espanyol,17951,Estadio Nuevo Los Cármenes,Alejandro Hernández,2.0,0.4,0,0
3038,38,Sun,2022-05-22,17:30:00,Elche,3–1,Getafe,17336,Estadio Manuel Martínez Valero,Guillermo Cuadra,2.5,1.3,3,1


In [11]:
scores_overall_df.dtypes

_week_                 int64
_day_                 object
_date_        datetime64[ns]
_time_                object
home                  object
score                 object
away                  object
attendance             Int64
venue                 object
referee               object
xg_home              float64
xg_away              float64
homegoal                int8
awaygoal                int8
dtype: object

In [12]:
def calc_missing_values(df):
    """
    Calculate percentage of NAs in dataframe
    :param df:
    :return:
    """
    num = df.isnull().sum()
    length = len(df)
    print("Percent NA")
    print(round(num/length, 2))

In [13]:
scores_overall_df.isna().sum()

_week_          0
_day_           0
_date_          0
_time_          0
home            0
score           0
away            0
attendance    488
venue           0
referee         0
xg_home         0
xg_away         0
homegoal        0
awaygoal        0
dtype: int64

In [14]:
scores_overall_df.to_pickle(os.path.join(os.path.realpath('../data/preprocessed/'), 'scores_overall.pkl'))